In [25]:
import keras
import mlflow
from mlflow.models import infer_signature

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from hyperopt import STATUS_OK,Trials,fmin,hp,tpe

In [26]:
data_df = pd.read_csv("https://raw.githubusercontent.com/MainakRepositor/Datasets/refs/heads/master/diabetes.csv", sep=",")

In [7]:
data_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### Data prepration:

In [27]:
X_data = data_df.drop(['Outcome'],axis=1).values
y_data = data_df[['Outcome']].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=42)

In [28]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [29]:
signature = infer_signature(X_train, y_train)

In [ ]:
def train_model(params, epochs, X_train, X_valid, X_test, y_train, y_valid, y_test):
    mean = np.mean(X_train, axis=0)
    var = np.var(X_train, axis=0)

    model=keras.Sequential(
        [
            keras.Input([X_train.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(  optimizer=keras.optimizers.SGD(
                    learning_rate=params["lr"],
                    momentum=params["momentum"]),
                    loss="mean_squared_error",
                    metrics=[keras.metrics.RootMeanSquaredError()]
                )
    
    # track training

    with mlflow.start_run(nested=True):
        model.fit(X_train, y_train,validation_data=(X_valid,y_valid),
                  epochs=epochs,
                  batch_size=64)
        
        eval_result=model.evaluate(X_valid,y_valid,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        mlflow.tensorflow.log_model(model,"model",signature=signature)
        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [32]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        X_train=X_train,
        y_train=y_train,
        X_valid=X_valid,
        y_valid=y_valid,
        X_test=X_test,
        y_test=y_test,
    )
    return result

In [33]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [34]:
mlflow.set_experiment("deiabetes-prediction")

with mlflow.start_run():
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2025/07/30 08:27:51 INFO mlflow.tracking.fluent: Experiment with name 'deiabetes-prediction' does not exist. Creating a new experiment.


  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

E0000 00:00:1753853271.474351    8893 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1753853271.498835    8893 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/3                                            

1/8 ━━━━━━━━━━━━━━━━━━━━ 1s 278ms/step - loss: 0.4890 - root_mean_squared_error: 0.6993
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.4018 - root_mean_squared_error: 0.6339 - val_loss: 0.3385 - val_root_mean_squared_error: 0.5818

Epoch 2/3                                            

1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.3085 - root_mean_squared_error: 0.5554
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3828 - root_mean_squared_error: 0.6187 - val_loss: 0.3150 - val_root_mean_squared_error: 0.5612

Epoch 3/3                                            

1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.3289 - root_mean_squared_error: 0.5735
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.3567 - root_mean_squared_error: 0.5973 - val_loss: 0.2898 - val_root_mean_squared_error: 0.5383

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3318 - root_mean_squared_error: 0.5761
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2898 

2025/07/30 08:27:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



...

Epoch 1/3                                                                      

1/8 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - loss: 0.4059 - root_mean_squared_error: 0.6371
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4869 - root_mean_squared_error: 0.6978 - val_loss: 0.4466 - val_root_mean_squared_error: 0.6683

Epoch 2/3                                                                      

1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.3520 - root_mean_squared_error: 0.5933
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4809 - root_mean_squared_error: 0.6935 - val_loss: 0.4409 - val_root_mean_squared_error: 0.6640

Epoch 3/3                                                                      

1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.4341 - root_mean_squared_error: 0.6589
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.4750 - root_mean_squared_error: 0.6892 - val_loss: 0.4350 - val_root_mean_squared_error: 0.6596

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.4763 - root_me

2025/07/30 08:28:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



100%|██████████| 4/4 [00:18<00:00,  4.52s/trial, best loss: 0.5382937788963318]

2025/07/30 08:28:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Best parameters: {'lr': np.float64(0.0003166058785022323), 'momentum': np.float64(0.8980089801343761)}
Best eval rmse: 0.5382937788963318


In [ ]:
## Inferencing

from mlflow.models import validate_serving_input

model_uri = "runs:/975ecdd4d2f646fc958311000bf7afb5/model"

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(X_test)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[ 2.93438554e-01],
       [ 1.81930184e-01],
       [ 5.58226764e-01],
       [ 7.05736339e-01],
       [ 1.45250931e-01],
       [ 1.54406384e-01],
       [ 6.63866937e-01],
       [ 1.02144098e+00],
       [ 9.35668498e-02],
       [-3.28742489e-02],
       [-1.00624308e-01],
       [ 1.10664010e-01],
       [ 4.99363810e-01],
       [ 5.33368051e-01],
       [ 3.04923683e-01],
       [ 3.83424401e-01],
       [ 1.32979542e-01],
       [ 2.40120552e-02],
       [ 4.42287296e-01],
       [ 2.69227326e-01],
       [ 4.56280634e-02],
       [ 7.65051916e-02],
       [-7.58702159e-02],
       [ 3.44886392e-01],
       [-1.55668054e-03],
       [ 2.50827193e-01],
       [ 6.53895885e-02],
       [ 4.20957774e-01],
       [ 1.71635255e-01],
       [ 1.49602428e-01],
       [ 7.82353103e-01],
       [ 8.60386640e-02],
       [ 2.79720396e-01],
       [ 7.42885709e-01],
       [ 2.13392809e-01],
       [ 3.54820341e-01],
       [ 7.20261991e-01],
       [ 3.62659603e-01],
       [ 2.3

In [ ]:

loaded_model = mlflow.pyfunc.load_model(model_uri=model_uri)


In [39]:
y_pred = loaded_model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [40]:
mlflow.register_model(model_uri=model_uri, name="diabetes_model")

Successfully registered model 'diabetes_model'.
2025/07/30 11:24:14 WARNING mlflow.tracking._model_registry.fluent: Run with id 975ecdd4d2f646fc958311000bf7afb5 has no artifacts at artifact path 'model', registering model based on models:/m-a75eea1c33104ccebb4cee5a7c72043d instead
Created version '1' of model 'diabetes_model'.


<ModelVersion: aliases=[], creation_timestamp=1753863854532, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1753863854532, metrics=[<Metric: dataset_digest=None, dataset_name=None, key='eval_rmse', model_id='m-a75eea1c33104ccebb4cee5a7c72043d', run_id='975ecdd4d2f646fc958311000bf7afb5', step=0, timestamp=1753853289546, value=0.5382937788963318>], model_id='m-a75eea1c33104ccebb4cee5a7c72043d', name='diabetes_model', params={'lr': '0.0003166058785022323', 'momentum': '0.8980089801343761'}, run_id='975ecdd4d2f646fc958311000bf7afb5', run_link=None, source='models:/m-a75eea1c33104ccebb4cee5a7c72043d', status='READY', status_message=None, tags={}, user_id=None, version=1>